In [ ]:
!pip install dask
!pip install 'fsspec>=0.3.3'


     |████████████████████████████████| 112kB 22.4MB/s 


In [ ]:
import os
import dask.dataframe as dd
import pandas as pd
from dask import delayed
import numpy as np


airports = ["ATL", "CLT", "DEN", "DFW", "EWR", "IAH", "JFK", "LAS", "LAX", "MCO", "MIA", "ORD", "PHX", "SEA", "SFO"]
flightCols = ["Origin", "Dest", "FlightDate", "Quarter", "Year", "Month", "DayofMonth", "DepTime", "DepDel15", "CRSDepTime", "DepDelayMinutes", "OriginAirportID", "DestAirportID", "ArrTime", "CRSArrTime", "ArrDel15", "ArrDelayMinutes"]
# "Date", "Airport"
weatherCols = ["Airport","Year","Month","Day","Hour","WindSpeedKmph", "WindDirDegree", "WeatherCode", "precipMM", "Visibility", "Pressure", "Cloudcover", "DewPointF", "WindGustKmph", "tempF", "WindChillF", "Humidity", "Time"]
weatherKeys = ["windspeedKmph", "winddirDegree", "weatherCode", "precipMM", "visibility", "pressure", "cloudcover", "DewPointF", "WindGustKmph", "tempF", "WindChillF", "humidity", "time"]

years = ['2016', '2017']

months = range(1, 13)
days   = range(0,32)
hours = range(0,24)

def hour(x):
    if len(str(x)) > 2:
        return int(str(x)[-3::-1][::-1]+"00")
    # elif len(str(x)) <2:
      # return 100
    return 100





fdf = dd.read_csv(os.path.join('drive','MyDrive','Data','flight-data','*','*','*.csv'),usecols=flightCols)
fdf = fdf.compute()
fdf= fdf.dropna()
fdf = fdf[fdf['Origin'].isin(airports)]
fdf = fdf[fdf['Dest'].isin(airports)]
fdf = fdf.astype({"DepTime":np.int64,"ArrTime":np.int64,"DepDelayMinutes":np.int64,"ArrDelayMinutes":np.int64,"DepDel15":np.int64,"ArrDel15":np.int64})
fdf = fdf.reset_index(drop=True)
fdf["H_dept"] =  pd.Series([hour(x)-100 for x in fdf["DepTime"]])
fdf["H_arr"] = pd.Series([hour(x)-100 for x in fdf["ArrTime"]])

fdf.shape
x = list(set(fdf["H_dept"]))
x.sort()
print(x)


[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300]


In [ ]:
def JTD(fpth,airport,year,month):

  wlist = []
  with open(fpth) as json_file:
    wdic = json.load(json_file)["data"]["weather"]
  
  for day in days:
    for hour in hours:
      tlist = [airport,year,month,day+1,hour]
      for key in weatherKeys:
        try :
          tlist.append(wdic[day]["hourly"][hour][key]) 
        except:
          pass
      if len(tlist) ==18:
        wlist.append(tlist)
  return pd.DataFrame(wlist,columns=weatherCols)


In [ ]:
import json 
pth= "drive/MyDrive/Data/weather"
# count = 0
wtlist = [ ]
for airport in airports:
  for year in years:
    for month in months:
      fpth = os.path.join(pth,airport,str(year)+"-"+str(month)+".json")
      wtlist.append(delayed(JTD)(fpth,airport,year,month).compute())
weatherDF = pd.concat(wtlist,ignore_index=True)
weatherDF = weatherDF.astype({"Year":np.int64,"Time":np.int64})

weatherDF.shape

In [ ]:
lst1 = ["Year","Month","DayofMonth","H_dept","Origin"]
lst2 = ["Year","Month","DayofMonth","H_arr","Dest"]
lst3 = ["Year","Month","Day","Time","Airport"]

InfoDF = fdf.merge(weatherDF, left_on=lst1, right_on=lst3)
print(InfoDF.shape)
InfoDF = InfoDF.merge(weatherDF, left_on=lst2, right_on=lst3, suffixes=("$Origin$", "$Dest$"))
InfoDF.shape
InfoDF.columns

In [ ]:
InfoDF.to_csv("drive/MyDrive/Data/Info.csv")

In [ ]:
* Dask
* isin function
* how to see list of names in coloum : print(df.columns.tolist())
* how to select list of coloums in pandas : df = pd.read_csv('data.csv', skipinitialspace=True, usecols=fields)
* dropna to drop coloumn with no values 
* shift + home = all text till begin
* shift + end = vice versa

In [ ]:

WindSpeedKmph	WindDirDegree	WeatherCode	precipMM	Visibility	Pressure	Cloudcover	DewPointF	WindGustKmph	tempF	WindChillF	Humidity	Time
0	11	318	176	0.1	10	1023	100	47	17	49	46	91	0
1	13	317	176	0.0	10	1023	100	44	22	46	42	92	100
2	14	315	122	0.0	10	1023	100	41	26	43	38	92	200
3	16	314	122	0.0	10	1023	100	38	30	40	33	93	300
4	17	314	122	0.0	10	1023	100	38	30	40	33	93	400
...	...	...	...	...	...	...	...	...	...	...	...	...	...
739	13	169	122	0.0	10	1015	96	54	20	60	60	82	1900
740	13	165	122	0.0	10	1015	95	54	20	60	59	82	2000
741	12	162	122	0.0	10	1015	95	54	20	59	59	83	2100
742	13	164	122	0.0	10	1015	92	54	21	59	58	83	2200
743	14	167	122	0.0	10	1015	89	53	23	58	57	84	2300

# New section